## «*Quien no se mueve, no siente las cadenas*».
### [Rosa de Luxemburgo](https://es.wikipedia.org/wiki/Rosa_Luxemburgo)

# ppi_SI_08_GeoPandas_SciPy

Actividad individual.

Utilice el dataset seleccionado en **SI02 Repositorio datos geoespaciales de Kaggle**.

Diligencie las casillas indicadas siguiendo las normas de estilo del PEP8.

# Instalar GeoPandas

GeoPandas no es una librería nativa de Python por lo que **requiere ser instalada antes de ser invocada**.


In [ ]:
# Instalar GeoPandas
!pip install geopandas

# Preparación del ejercicio

*   Importe las librerías requeridas
*   Lea el dataset a utilizar desde una url

Sugerencia: suba el dataset a Googledrive y luego publíquelo como página web. El siguiente enlace le indica cómo hacerlo: [Cómo publicar archivos de Documentos, Hojas de cálculo, Presentaciones y Formularios de Google](https://support.google.com/docs/answer/183965?hl=es-419&co=GENIE.Platform%3DDesktop).

Nota: si lo desea puede utilizar otro método para obtener la url del dataset.



In [ ]:
# Importar las librerías necesarias
import numpy as np
import pandas as pd
import matplotlib as mpl
import geopandas as gpd

# Cordial saludo profe,
# Inicialmente mi dataset de kaggle era de más\
# de 100mil columnas, debido a que el dataset\
# era tan grande, públique una versión reducida a\
# 10mil columnas!

# Conocer la versiones de las librerías instaladas
print(f"La versión instalada de NumPy es: {np.__version__}\n")
print(f"La versión instalada de Pandas es: {pd.__version__}\n")
print(f"La versión instalada de Matplotlib es: {mpl.__version__}\n")
print(f"La versión instalada de GeoPandas es: {gpd.__version__}\n")

# Guardar en la variable 'ruta' la url del dataset
ruta = ('https://docs.google.com/spreadsheets/d/e/'
        '2PACX-1vS8X_0a4e7CkOgPBgZWpDRKQhk82Eu4goC'
        '1-Jgbx7JuaBOm6x67gQx162jy6dxnJGm5q8FmHLik'
        'D4Do/pub?output=csv')

# Cargar el dataset a partir de la ruta establecida con andas
data = pd.read_csv(ruta, sep=',')

# Se crea el GeoDataFrame con GeoPandas
geo_dataframe = gpd.GeoDataFrame(
    data,
    geometry=gpd.points_from_xy(data['LONGITUDE'], data['LATITUDE'])
    )

# Verificar la lectura del dataset
print(geo_dataframe)


## '00.  Mostrar datos

Ubique en un mapa las coordenadas de los puntos contenidos en el dataset.

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd

# Se genera un gráfico tipo plot
ax = geo_dataframe.plot(
    marker='o',
    markersize=3,
    color='blue',
    figsize=(12, 10)
    )

# Se define el título y ejes del gráfico
ax.set_title("Ciudades en el mapa")
ax.set_xlabel("Longitud")
ax.set_ylabel("Latitud")

# Se muestra el mapa
plt.show()


## '01. Seleccionar datos

Ubique en un mapa los cinco puntos más alejados del centro del dataset.

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd

# Se calcula el punto central del dataset
central_point = geo_dataframe.geometry.unary_union.centroid

# Se calcula la distancia del punto central
geo_dataframe['center_distance'] = geo_dataframe.distance(central_point)

# Se genera una copia del dataframe
geo_dataframe_copy = geo_dataframe.copy()

# Se seleccionan los 5 puntos más alejados
farthest_points = geo_dataframe_copy.sort_values(
    by='center_distance',
    ascending=False
    ).head(5)

# Se muestran todos los puntos del gráfico
ax = geo_dataframe.plot(
    marker='o',
    markersize=3,
    color='blue',
    figsize=(12, 10)
    )

# Se pintan los 5 puntos más lejanos del centro del dataset
farthest_points.plot(ax=ax, color='red', marker='*', markersize=60)

# Se pinta el centro del dataset
central_point_df = gpd.GeoDataFrame({'geometry': [central_point]})
central_point_df.plot(ax=ax, color='black', marker='^', markersize=60)

# Se define el título y ejes del gráfico
ax.set_title("Cinco ciudades más alejadas del centro del mapa")
ax.set_xlabel("Longitud")
ax.set_ylabel("Latitud")

# Se muestra el gráfico
plt.show()


## '02. Distancia euclidiana

Calcule la distancia euclideana para los puntos del dataset. Luego indique el promedio de las distancias, la distancia mínima y la distancia máxima entre dos puntos. Muestre el par de coordenadas cuya distancia es mínima y el par de coordenadas cuya distancia es mínima.

In [ ]:
import numpy as np
import scipy.spatial.distance as dst

# Se crea una copia del DataFrame.
data_02 = geo_dataframe.copy()

# Se genera una matriz de puntos del dataset.
raw_points_02 = data_02.geometry.apply(lambda geom: (geom.x, geom.y)).tolist()
points_02 = np.array(raw_points_02)

# Se calculan las distancias euclidianas entre todos los puntos.
euclidean_distance = dst.cdist(points_02, points_02, 'euclidean')

# Se rellena la diagonal para no contar puntos iguales.
np.fill_diagonal(euclidean_distance, np.nan)

# Se calcula la distancia promedio.
mean_value_02 = np.nanmean(euclidean_distance)
print(f'La distancia euclidiana promedio es: {mean_value_02}\n')

# Se calcula la distancia maxima.
max_value_02 = np.nanmax(euclidean_distance)
print(f'La distancia euclidiana máxima es: {max_value_02}\n')

# Se calcula la distancia minima.
min_value_02 = np.nanmin(euclidean_distance)
print(f'La distancia euclidiana mínima es: {min_value_02}\n')

# Se encuentran los puntos que corresponden a la mayor distancia euclidiana.
max_idx_02 = np.unravel_index(
    np.nanargmax(euclidean_distance),
    euclidean_distance.shape
    )
print("Puntos que generan la distancia euclidiana máxima:")
print(data_02.iloc[max_idx_02[0]][['Name', 'geometry']], '\n')
print(data_02.iloc[max_idx_02[1]][['Name', 'geometry']], '\n')

# Se encuentran los puntos que corresponden a la menor distancia euclidiana.
min_idx_02 = np.unravel_index(
    np.nanargmin(euclidean_distance),
    euclidean_distance.shape
    )
print("Puntos que generan la distancia euclidiana mínima:")
print(data_02.iloc[min_idx_02[0]][['Name', 'geometry']], '\n')
print(data_02.iloc[min_idx_02[1]][['Name', 'geometry']], '\n')


## '03. Distancia de Manhattan

Calcule la distancia de Manhattan para los puntos del dataset. Luego indique el promedio de las distancias, la distancia mínima y la distancia máxima entre dos puntos. Muestre el par de coordenadas cuya distancia es mínima y el par de coordenadas cuya distancia es mínima.

In [ ]:
import numpy as np
import scipy.spatial.distance as dst

# Se crea una copia del DataFrame.
data_03 = geo_dataframe.copy()

# Se genera una matriz de puntos del dataset.
raw_points_03 = data_03.geometry.apply(lambda geom: (geom.x, geom.y)).tolist()
points_03 = np.array(raw_points_03)

# Se calculan las distancias manhattan entre todos los puntos.
manhattan_distance = dst.cdist(points_03, points_03, 'cityblock')

# Se rellena la diagonal para no contar puntos iguales.
np.fill_diagonal(manhattan_distance, np.nan)

# Se calcula la distancia manhattan promedio.
mean_value_03 = np.nanmean(manhattan_distance)
print(f'La distancia manhattan promedio es: {mean_value_03}\n')

# Se calcula la distancia maxima.
max_value_03 = np.nanmax(manhattan_distance)
print(f'La distancia manhattan máxima es: {max_value_03}\n')

# Se calcula la distancia minima.
min_value_03 = np.nanmin(manhattan_distance)
print(f'La distancia manhattan mínima es: {min_value_03}\n')

# Se encuentran los puntos que corresponden a la mayor distancia manhattan.
max_idx_03 = np.unravel_index(
    np.nanargmax(manhattan_distance),
    manhattan_distance.shape
    )
print("Puntos que generan la distancia manhattan máxima:")
print(data_03.iloc[max_idx_03[0]][['Name', 'geometry']], '\n')
print(data_03.iloc[max_idx_03[1]][['Name', 'geometry']], '\n')

# Se encuentran los puntos que corresponden a la menor distancia manhattan.
min_idx_03 = np.unravel_index(
    np.nanargmin(manhattan_distance),
    manhattan_distance.shape
    )
print("Puntos que generan la distancia manhattan mínima:")
print(data_03.iloc[min_idx_03[0]][['Name', 'geometry']], '\n')
print(data_03.iloc[min_idx_03[1]][['Name', 'geometry']], '\n')


## '04. Distancia de coseno

Calcule la distancia de coseno para los puntos del dataset. Luego indique el promedio de las distancias, la distancia mínima y la distancia máxima entre dos puntos. Muestre el par de coordenadas cuya distancia es mínima y el par de coordenadas cuya distancia es mínima.

In [ ]:
import numpy as np
import scipy.spatial.distance as dst

# Se crea una copia del DataFrame.
data_04 = geo_dataframe.copy()

# Se genera una matriz de puntos del dataset.
raw_points_04 = data_04.geometry.apply(lambda geom: (geom.x, geom.y)).tolist()
points_04 = np.array(raw_points_04)

# Se calculan las distancias coseno entre todos los puntos.
cosine_distance = dst.cdist(points_04, points_04, 'cosine')

# Se rellena la diagonal para no contar puntos iguales.
np.fill_diagonal(cosine_distance, np.nan)

# Se calcula la distancia coseno promedio.
mean_value_04 = np.nanmean(cosine_distance)
print(f'La distancia coseno promedio es: {mean_value_04}\n')

# Se calcula la distancia maxima.
max_value_04 = np.nanmax(cosine_distance)
print(f'La distancia coseno máxima es: {max_value_04}\n')

# Se calcula la distancia minima.
min_value_04 = np.nanmin(cosine_distance)
print(f'La distancia coseno mínima es: {min_value_04}\n')

# Se encuentran los puntos que corresponden a la mayor distancia coseno.
max_idx_04 = np.unravel_index(
    np.nanargmax(cosine_distance),
    cosine_distance.shape
    )
print("Puntos que generan la distancia coseno máxima:")
print(data_04.iloc[max_idx_04[0]][['Name', 'geometry']], '\n')
print(data_04.iloc[max_idx_04[1]][['Name', 'geometry']], '\n')

# Se encuentran los puntos que corresponden a la menor distancia coseno.
min_idx_04 = np.unravel_index(
    np.nanargmin(cosine_distance),
    cosine_distance.shape
    )
print("Puntos que generan la distancia coseno mínima:")
print(data_04.iloc[min_idx_04[0]][['Name', 'geometry']], '\n')
print(data_04.iloc[min_idx_04[1]][['Name', 'geometry']], '\n')


## '05. Triangulación de Delaunay

Aplique la [Triangulación de Delaunay](https://en.wikipedia.org/wiki/Delaunay_triangulation) para los puntos que se encuentran en el dataset.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial as spt

# Se crea uan copia del dataframe.
data_05 = geo_dataframe.copy().geometry

# Se crea un arreglo con los puntos del mapa.
points_05 = np.array(data_05.apply(lambda geom: (geom.x, geom.y)).tolist())

# Se generan los triangulos de Delaunay.
triangles = spt.Delaunay(points_05).simplices

# Se establece el tamaño de la figura.
plt.figure(figsize=(15, 8))

# Se pintan los triangulos de Delaunay.
plt.triplot(points_05[:, 0], points_05[:, 1], triangles)

# Se pintan los puntos en el mapa.
plt.scatter(points_05[:, 0], points_05[:, 1], s=5, color='r')

# Se define el título y ejes del gráfico.
plt.title("Triangulación Delaunay de ciudades en el mapa")
plt.xlabel("Longitud")
plt.ylabel("Latitud")

# Se muestra el gráfico.
plt.show()


## '06. Casco convexo

Calcule y muestre el casco convexo para los puntos que se encuentran en el dataset.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial as spt

# Se crea uan copia del dataframe.
data_06 = geo_dataframe.copy().geometry

# Se crea un arreglo con los puntos del mapa.
points_06 = np.array(data_06.apply(lambda geom: (geom.x, geom.y)).tolist())

# Se genera el casco convexo
casco = spt.ConvexHull(points_06).simplices

# Se establece el tamaño de la figura.
plt.figure(figsize=(15, 8))

# Se pinta el casco convexo.
for s in casco:
    plt.plot(points_06[s, 0], points_06[s, 1], 'k-')

# Se pintan los puntos en el mapa.
plt.scatter(points_06[:, 0], points_06[:, 1], s=5, color='r')

# Se define el título y ejes del gráfico.
plt.title("Casco convexo de ciudades en el mapa")
plt.xlabel("Longitud")
plt.ylabel("Latitud")

# Se muestra el gráfico.
plt.show()


## '07. Sumatoria de mínimos cuadrados

Calcule la sumatoria de mínimos cuadrados para los puntos que se encuentran en el dataset.

In [ ]:
import numpy as np
import geopandas as gpd
from scipy.optimize import least_squares


def error_function(params, points, values):
    """La función define la diferencia entre los valores observados y los\
    valores predichos.

    La función debe recibir un vector de puntos y valores del mismo tamaño.
    """

    a, b = params
    predicted = a * points.x + b
    residuals = predicted - values
    return residuals


# se crea una copia del dataframe.
data_07 = geo_dataframe.copy()

# Se extraen los puntos y valores.
points_07 = data_07.geometry
values_07 = data_07['Population']

# Se encuentran los valores óptimos que minimizan la suma de los cuadrados.
optimals_07 = least_squares(
    error_function,
    [1.0, 0.0],
    args=(points_07, values_07)
    )
optimal_a_07, optimal_b_07 = optimals_07.x

# Se imprimen los valores optimos.
print(f"Valor óptimo de a: {optimal_a_07}\n")
print(f"Valor óptimo de b: {optimal_b_07}\n")

# Se imprime la sumatoria de mínimos cuadrados.
predicted_values_07 = optimal_a_07 * points_07.x + optimal_b_07
result_07 = np.sum((predicted_values_07 - values_07) ** 2)

print(f"Suma de mínimos cuadrados basada en la población\
: {result_07}")


## '08. Cluster

Calcule el agrupamiento (cluster) de nivel tres para los puntos que se encuentran en el dataset.

In [ ]:
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist, squareform
from scipy.cluster.hierarchy import linkage, fcluster

# Se crea una copia del dataframe.
data_07 = geo_dataframe.copy()

# Se genera una matriz de puntos del dataset.
raw_points_07 = data_07.geometry.apply(lambda geom: (geom.x, geom.y)).tolist()
points_07 = np.array(raw_points_07)

# Se calcula la distancia euclidiana entre los puntos.
euclidean_07 = cdist(points_07, points_07, 'euclidean')

# Se generan los clusters.
square_euclidean_07 = squareform(euclidean_07)
linkage_07 = linkage(square_euclidean_07, method='ward')
clusters_07 = fcluster(linkage_07, t=3, criterion='maxclust')

# Se guarda en el datafram los clusters.
data_07['Cluster'] = clusters_07

# Se define el número de colores a utilizar para los clusters.
num_clusters_07 = max(clusters_07)
colors_07 = plt.cm.viridis(np.linspace(0, 1, num_clusters_07))

# Se crea una figura.
fig, ax = plt.subplots(1, 1, figsize=(15, 12))

# Se dibuja el GeoDataFrame utilizando colores basados en los clusters.
for cluster_id, color in zip(range(1, num_clusters_07 + 1), colors_07):
    data_07[data_07['Cluster'] == cluster_id].plot(
        ax=ax,
        color=color,
        markersize=7,
        label=f'Cluster {cluster_id}'
        )

# Se agrega una leyenda.
ax.legend()

# Se define el título y los ejes.
plt.title('Agrupamiento de nivel 3')
plt.xlabel('Longitud')
plt.ylabel('Latitude')

# Se muestra el gráfico.
plt.show()
